In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import os

In [5]:
os.chdir('..')

In [7]:
os.chdir('10_Data/30_Extracted Tweets by Disaster/2019 B Test')

In [6]:
os.getcwd()

'C:\\Users\\Vishaal\\Documents\\GitHub\\TREC_Distributed_Machine_Learning\\TREC'

In [8]:
df = pd.read_csv('fire_TREC_2019_B_test.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,ID,Tweet,Event,Source,User_Language,Event_Decrption,Categories,Priority
0,0,0,1134239576111771648,"Tolkoâ€™s Athabasca division in Slave Lake, Al...",albertaWildfires2019,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",NaN,The 2019 Alberta wildfires were described by N...,"['Location', 'News']",High
1,1,1,1131161667398774784,'Heartbroken' Alberta woman describes wildfire...,albertaWildfires2019,"<a href=""https://www.awayshare.com/"" rel=""nofo...",NaN,The 2019 Alberta wildfires were described by N...,['Irrelevant'],Low
2,2,2,1134148540719017984,There is so much smoke from the high level fir...,albertaWildfires2019,"<a href=""http://twitter.com/download/iphone"" r...",NaN,The 2019 Alberta wildfires were described by N...,['Irrelevant'],Low
3,3,3,1133705772246786049,Waking to smoky sunrise in Calgary today - eer...,albertaWildfires2019,"<a href=""http://twitter.com/download/android"" ...",NaN,The 2019 Alberta wildfires were described by N...,['Irrelevant'],Low
4,4,4,1134018172309295104,BREAKING: Hundreds of evacuees in Calling Lake...,albertaWildfires2019,"<a href=""https://about.twitter.com/products/tw...",NaN,The 2019 Alberta wildfires were described by N...,"['Location', 'Factoid', 'Hashtags', 'News']",Critical


In [9]:
'''
    Converting tweet column to str
'''
df['Tweet'] = df['Tweet'].astype('str')

In [10]:
'''
    Generalise process to all files. Maybe later
'''
event_type = ['Floods', 'Earthquake', 'Bushfire', 'Bombings', 'Tornado', 'Attack', 'SchoolShooting', 'typhoon' ]

In [11]:
'''
    Tokenize each tweet into words. Note we haven't yet removed stop words
'''
token_array = []
for tweet in df['Tweet']:
    token_tweet = word_tokenize(tweet)
    token_array.append(token_tweet)
                       
token_array[1]

["'Heartbroken",
 "'",
 'Alberta',
 'woman',
 'describes',
 'wildfire',
 'evacuation',
 "'Heartbroken",
 "'",
 'Alberta',
 'woman',
 'describes',
 'wildfire',
 'evacuation',
 '&',
 'lt',
 ';',
 'p',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 'img',
 'src=',
 "''",
 'https',
 ':',
 '//t.co/6JN9HiOCaK',
 "''",
 '/',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 '/p',
 '&',
 'gt',
 ';',
 'The',
 'town',
 'of',
 'High',
 'Level',
 ',',
 'Alta.',
 ',',
 'and',
 'a',
 'nearby',
 'https',
 ':',
 '//t.co/EEdMYc3EYi',
 'https',
 ':',
 '//t.co/PPApbrj1v1']

In [12]:
'''
    Will remove stop words from tweet. We still have to look into removing punctuation marks.
'''
stop_words=set(stopwords.words("english"))
filtered_token_array=[]
for tweet in token_array:
    filtered_tweet = []
    for word in tweet:
        if word not in stop_words:
            filtered_tweet.append(word)
    filtered_token_array.append(filtered_tweet)
    
filtered_token_array[1]

["'Heartbroken",
 "'",
 'Alberta',
 'woman',
 'describes',
 'wildfire',
 'evacuation',
 "'Heartbroken",
 "'",
 'Alberta',
 'woman',
 'describes',
 'wildfire',
 'evacuation',
 '&',
 'lt',
 ';',
 'p',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 'img',
 'src=',
 "''",
 'https',
 ':',
 '//t.co/6JN9HiOCaK',
 "''",
 '/',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 '/p',
 '&',
 'gt',
 ';',
 'The',
 'town',
 'High',
 'Level',
 ',',
 'Alta.',
 ',',
 'nearby',
 'https',
 ':',
 '//t.co/EEdMYc3EYi',
 'https',
 ':',
 '//t.co/PPApbrj1v1']

In [13]:
'''
    We will now do stemming. This is the process of removing different forms of the same word and will
    resort to the root word. For example, connection, connected, connecting word reduce to a common 
    word "connect".
'''
ps = PorterStemmer()
stemmed_array=[]
for tweet in filtered_token_array:
    stemmed_tweet = []
    for word in tweet:
        stemmed_tweet.append(ps.stem(word))
    stemmed_array.append(stemmed_tweet)
    
stemmed_array[1]

["'heartbroken",
 "'",
 'alberta',
 'woman',
 'describ',
 'wildfir',
 'evacu',
 "'heartbroken",
 "'",
 'alberta',
 'woman',
 'describ',
 'wildfir',
 'evacu',
 '&',
 'lt',
 ';',
 'p',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 'img',
 'src=',
 "''",
 'http',
 ':',
 '//t.co/6jn9hiocak',
 "''",
 '/',
 '&',
 'gt',
 ';',
 '&',
 'lt',
 ';',
 '/p',
 '&',
 'gt',
 ';',
 'the',
 'town',
 'high',
 'level',
 ',',
 'alta.',
 ',',
 'nearbi',
 'http',
 ':',
 '//t.co/eedmyc3eyi',
 'http',
 ':',
 '//t.co/ppapbrj1v1']

In [14]:
'''
    We now do lemmatization. This is like stemming but more effective apparently as it does a dictionary lookup. For 
    instance a relation between the words good and better may be made in lemmatisation but not in stemming.
    
    Lemmatization is much better from a cursory look. Words like earthquake are being cut down to earthquak 
    when using stemming. 
'''
lem = WordNetLemmatizer()
stem = PorterStemmer()

lemmatized_array=[]
for tweet in filtered_token_array:
    lemmatized_tweet = []
    for word in tweet:
        lemmatized_tweet.append(lem.lemmatize(word,'v'))
    lemmatized_array.append(lemmatized_tweet)

lemmatized_array_join = []
for element in lemmatized_array:
    lemmatized_array_join.append(''.join(element))

In [15]:
'''
    DTM to get bag of words
'''

cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1))
text_counts= cv.fit_transform(lemmatized_array_join)
text_counts_dense = text_counts.todense()

In [16]:
'''
    DTM to get TF-IDF features
'''

tf=TfidfVectorizer()
text_tf= tf.fit_transform(lemmatized_array_join)
text_tf_dense = text_tf.todense()

In [17]:
'''
    Converting TF-IDF to list of lists. Then we play around with the datatypes to get features as a dense
    list of numpy arrays l_features. We also get the actual word names that are used as features.
'''
words = tf.get_feature_names()
M = text_tf.tolil()
l_features = []
for i in range(M.shape[0]):
    l_features.append(np.array(M[i].todense())[0])

In [18]:
'''
    Converting to dataframe
'''
df_features = pd.DataFrame(l_features)

In [19]:
'''
    Combining both the dataframes - we now have the TF-IDF features and all the other stuff we had before
'''
df_combined = pd.concat([df,df_features], axis=1)

In [20]:
'''
    List of columns with original columns and word features
'''
cols = list(df.columns) + words

In [21]:
'''
    Rename columns in original dataframe with new column names
'''
df_combined = df_combined.rename(columns={x:y for x,y in zip(df_combined.columns,cols)})

In [22]:
'''
    Creating a function to the above to all the files...
'''
def preProcess(file):
    df = pd.read_csv(file+'_TREC_2019_B_test.csv')
    df['Tweet'] = df['Tweet'].astype('str')
    
    token_array = []
    for tweet in df['Tweet']:
        token_tweet = word_tokenize(tweet)
        token_array.append(token_tweet)
        
    stop_words=set(stopwords.words("english"))
    filtered_token_array=[]
    for tweet in token_array:
        filtered_tweet = []
        for word in tweet:
                if word not in stop_words:
                    filtered_tweet.append(word)
        filtered_token_array.append(filtered_tweet)
        
    lem = WordNetLemmatizer()
    stem = PorterStemmer()

    lemmatized_array=[]
    for tweet in filtered_token_array:
        lemmatized_tweet = []
        for word in tweet:
            lemmatized_tweet.append(lem.lemmatize(word,'v'))
        lemmatized_array.append(lemmatized_tweet)
    
    lemmatized_array_join = []
    for element in lemmatized_array:
        lemmatized_array_join.append(''.join(element))
        
    return (lemmatized_array_join)
    
    
def tfidf(file, corpus):
    df = pd.read_csv(file+'_TREC_2019_B_test.csv')
    tf=TfidfVectorizer()
    text_tf= tf.fit_transform(corpus)
    text_tf_dense = text_tf.todense()
    
    words = tf.get_feature_names()
    M = text_tf.tolil()
    l_features = []
    for i in range(M.shape[0]):
        l_features.append(np.array(M[i].todense())[0])
        
    df_features = pd.DataFrame(l_features)
    
    df_combined = pd.concat([df,df_features], axis=1)
    
    cols = list(df.columns) + words
    
    df_combined_1 = df_combined.rename(columns={x:y for x,y in zip(df_combined.columns,cols)})
    
    return (df_combined_1)

In [23]:
event_type = ['Flood', 'Earthquake', 'fire', 'Shooting', 'cyclone' ]

for disaster in event_type:
    feature = preProcess(disaster)
    df = tfidf(disaster, feature)
    df.to_csv(disaster+'_tfidf_features_2019_B_test.csv')
